# Imports

In [ ]:
# Import python packages
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.cortex import complete
session = get_active_session()
root = Root(session)

# 1. Structured Data (Cortex Analyst)
## 1.1 Overview

By now we have turned various unstructured data sources into structured information and can combine them with existing structured information.  

Our final goal is to create a [Semantic View](https://docs.snowflake.com/en/user-guide/views-semantic/overview) that accurately describes the available data from a business point of view and is used by [Cortex Analyst](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst) to provide high quality Text-To-SQL accuracy.

In [ ]:
SHOW TABLES IN SCHEMA SI_THE_FOOD_AND_BEVERAGE_COMPANY;

## 1.2 Dynamic Literal Retrieval

Imagine a question like this:
> _"What was the total revenue for Belgian dark chocolate 70 percent?"_  

This question clearly defines the product the user is looking for but in our database the product is actually named __Belgian Dark Chocolate 70%__. Cortex Analyst must be smart enough to find the correct product name when generating a SQL query.

A simple solution would be to provide sample values in the semantic view but this approach does not scale for high cardinality columns with potentially frequently changing values. For example the DIM_PRODUCTS table has 130 different products and updates frequently as new products are created.

If you have more than 10 unique values in a column, it is recommended to create a [Cortex Search Service](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview) and integrate the service into the Semantic View as described [here](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/cortex-analyst-search-integration). The Search Service will automatically update the available values at specified intervals defined in the __TARGET_LAG__ variable.  
That means, once you created the search service new literals will be available automatically and there is no need for any custom pipeline.

In [ ]:
-- Example of a high cardinality column that is changing frequently
SELECT 
    DISTINCT PRODUCT_NAME, PRODUCT_ID
FROM
    DIM_PRODUCTS;

In [ ]:
-- Create a Cortex Search Service for product names
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS _LITERAL_SEARCH_PRODUCT_NAME
  ON PRODUCT_NAME
  ATTRIBUTES PRODUCT_ID
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT PRODUCT_NAME, PRODUCT_ID
  FROM DIM_PRODUCTS
);

In [ ]:
-- Create a Cortex Search Service for product categories
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS _LITERAL_SEARCH_PRODUCT_HIERARCHY_1
  ON CATEGORY_HIER_1_NAME
  ATTRIBUTES CATEGORY_HIER_1_ID
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT CATEGORY_HIER_1_NAME, CATEGORY_HIER_1_ID
  FROM DIM_PRODUCT_HIERARCHY
);

In [ ]:
-- Create a Cortex Search Service for product subcategories
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS _LITERAL_SEARCH_PRODUCT_HIERARCHY_2
  ON CATEGORY_HIER_2_NAME
  ATTRIBUTES CATEGORY_HIER_2_ID
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT CATEGORY_HIER_2_NAME, CATEGORY_HIER_2_ID
  FROM DIM_PRODUCT_HIERARCHY
);

In [ ]:
-- Create a Cortex Search Service for countries
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS _LITERAL_SEARCH_CUSTOMER_COUNTRY
  ON COUNTRY
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT COUNTRY
  FROM DIM_CUSTOMERS
);

In [ ]:
-- Create a Cortex Search Service for customer names
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS _LITERAL_SEARCH_CUSTOMER_NAME
  ON NAME
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT NAME
  FROM DIM_CUSTOMERS
);

## 1.3 Test Dynamic Literal Retrieval

We can quickly test the created Search Services by using the [Python-API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service#python-api).

The API will return the most relevant products and correctly identifies __Belgian Dark Chocolate 70%__ as the product from the user's question.

In [ ]:
# Question
question = 'What was the total revenue for Belgian dark chocolate 70 percent?'
model = 'mistral-large2'

# Connect to Search Service
search_service = (root
  .databases["AI_DEVELOPMENT"]
  .schemas["SI_THE_FOOD_AND_BEVERAGE_COMPANY"]
  .cortex_search_services["_LITERAL_SEARCH_PRODUCT_NAME"]
)

# Search relevant documents
search_results = search_service.search(
  query=question,
  columns=['PRODUCT_NAME', 'PRODUCT_ID'],
  limit=3
)

# Display top 3 products related to user question
pd.DataFrame(search_results.results)

## 1.4 Create a Semantic View

Semantic Views can be created visually using the [Cortex Analyst Semantic View Generator](https://docs.snowflake.com/en/user-guide/views-semantic/ui).  
Alternatively you can create them using the [CREATE SEMANTIC](https://docs.snowflake.com/en/sql-reference/sql/create-semantic-view) SQL command.

__Why Semantic Views?__  
Semantic views address the mismatch between how business users describe data and how it’s stored in database schemas.
They are a way to surface your data with rich context, enabling AI-powered analytics and more intuitive querying.

Snowflake Cortex can use this context to translate natural language questions into SQL for that specific semantic view. So instead of asking an LLM to guess what cust_id means, it will know it’s "unique identifier for a customer".

It also allows to define standard business metrics as SQL expression, such as:
```yaml
metrics:
  - name: PROFIT
    synonyms:
      - margin
      - gross_profit
      - profit_margin
    description: ' Total profit calculated as revenue minus cost of goods sold'
    expr: SUM(FACT_TRANSACTIONS.QUANTITY * (DIM_PRODUCTS.PRODUCT_UNIT_PRICE - DIM_PRODUCTS.PRODUCT_UNIT_COGS))
```

In [ ]:
-- Create a Semantic View via SQL
create or replace semantic view AI_DEVELOPMENT.SI_THE_FOOD_AND_BEVERAGE_COMPANY.SALES_MODEL tables (
    DIM_CUSTOMERS primary key (CUSTOMER_ID) with synonyms =(
        'CUSTOMERS',
        'CLIENTS',
        'CONSUMERS',
        'PATRONS',
        'CLIENT_BASE',
        'CUSTOMER_BASE'
    ) comment = 'Master customer database containing demographic profiles, geographic locations, and personal characteristics of all customers who have purchased food and beverage products. Use this table to analyze customer segments, demographics, age groups, and geographic distribution. Essential for customer analytics, market segmentation, and personalized marketing strategies.',
    DIM_DATES primary key (DATE_KEY) with synonyms =(
        'DIM_DATE',
        'CALENDAR',
        'DATE_DIMENSION',
        'TIME_DIM',
        'CALENDAR_TABLE',
        'DATE_TABLE',
        'TIME_TABLE',
        'CALENDAR_DIM',
        'DATE_DIM',
        'CALENDAR_INFO'
    ) comment = 'Comprehensive calendar dimension providing date intelligence for time-based analysis across all business operations. Contains every date with pre-calculated time periods (weeks, months, quarters, years), seasonality indicators, and business calendar attributes. Use for trending analysis, period-over-period comparisons, seasonal patterns, and time-based filtering of sales, inventory, and customer data.',
    DIM_PLATFORMS primary key (PLATFORM_ID) with synonyms =('platforms') comment = 'This table stores information about various platforms, with each platform uniquely identified by a platform ID. The platform name provides a descriptive label for each platform, allowing for easy identification and reference.',
    DIM_PRODUCTS primary key (PRODUCT_ID) with synonyms =(
        'products',
        'product_list',
        'product_dimension',
        'product_master',
        'product_catalog'
    ) comment = 'Complete product catalog containing all food and beverage items with pricing, categorization, cost structure, and product tier classifications (Premium, Standard, Budget). Essential for product performance analysis, pricing strategies, profitability analysis, and inventory management decisions. Links to product hierarchy for category-based analysis.',
    DIM_PRODUCT_HIERARCHY primary key (CATEGORY_HIER_1_ID, CATEGORY_HIER_2_ID) with synonyms =(
        'product categories',
        'product classification',
        'product hierarchy table',
        'product levels',
        'category structure',
        'product taxonomy',
        'product categorization',
        'product grouping'
    ) comment = 'Two-level product classification system organizing products into business categories (Beverages, Confectionery, Food Products, Health & Nutrition, Pet Care) and subcategories (Coffee, Chocolate bars, Dairy, etc.). Enables category management, market analysis, and strategic product portfolio decisions.',
    DIM_SUPPLIERS primary key (SUPPLIER_ID) with synonyms =(
        'vendors',
        'suppliers_list',
        'supplier_table',
        'supplier_info',
        'supplier_data',
        'supplier_details'
    ) comment = 'This table stores information about the suppliers of a company, including their unique identifier, name, location details, and contact person.',
    FACT_DAILY_STOCK_LEVELS with synonyms =(
        'STOCK_LEVELS',
        'DAILY_INVENTORY',
        'PRODUCT_STOCK_STATUS',
        'INVENTORY_LEVELS',
        'STOCK_STATUS_REPORT',
        'DAILY_PROD_STOCK',
        'PRODUCT_AVAILABILITY',
        'STOCK_LEVEL_REPORT'
    ) comment = 'This view provides a daily snapshot of stock levels for each product, including the date, product ID, daily stock change, and available stock. It aggregates supplier ddeliveries and customer transactions to calculate the daily stock change, and then uses a running total to determine the available stock for each product on each date.',
    FACT_SUPPLIER_DELIVERIES with synonyms =(
        'supplier_deliveries',
        'deliveries',
        'supplier_shipments',
        'product_shipments',
        'inventory_deliveries',
        'supply_chain_data'
    ) comment = 'This table stores historical data on deliveries made by suppliers to the company, including the product ID, supplier ID, quantity delivered, and the date of delivery, allowing for tracking and analysis of supplier performance and inventory management.',
    FACT_TRANSACTIONS with synonyms =(
        'transactions',
        'sales',
        'orders',
        'purchases',
        'customer_purchases',
        'product_sales',
        'platform_transactions',
        'daily_transactions'
    ) comment = 'Complete sales transaction history capturing every customer purchase across all platforms and products. The foundation for revenue analysis, sales performance tracking, customer behavior analysis, and business growth metrics. Includes quantity sold, platform attribution, customer engagement metrics, and transactional detail for comprehensive business intelligence.',
    FACT_CUSTOMER_REVIEWS with synonyms =(
        'customer_reviews',
        'customer_feedback',
        'product_ratings',
        'customer_observations',
        'product_comments',
        'customer_opinions',
        'review_data',
        'customer_insights'
    ) comment = 'Customer feedback and sentiment analysis data providing insights into product satisfaction, brand perception, and customer experience quality. Contains AI-powered sentiment scores for different aspects (product quality, pricing, brand perception) enabling reputation management, product improvement identification, and customer satisfaction trending analysis.'
) relationships (
    PRODUCTS_HIERARCHY as DIM_PRODUCTS(CATEGORY_HIER_1_ID, CATEGORY_HIER_2_ID) references DIM_PRODUCT_HIERARCHY(CATEGORY_HIER_1_ID, CATEGORY_HIER_2_ID),
    STOCK_DATES as FACT_DAILY_STOCK_LEVELS(DATE_KEY) references DIM_DATES(DATE_KEY),
    STOCK_PRODUCTS as FACT_DAILY_STOCK_LEVELS(PRODUCT_ID) references DIM_PRODUCTS(PRODUCT_ID),
    SUPPLIER_DATES as FACT_SUPPLIER_DELIVERIES(DATE_KEY) references DIM_DATES(DATE_KEY),
    SUPPLIER_DIMENSIONS as FACT_SUPPLIER_DELIVERIES(SUPPLIER_ID) references DIM_SUPPLIERS(SUPPLIER_ID),
    SUPPLIER_PRODUCTS as FACT_SUPPLIER_DELIVERIES(PRODUCT_ID) references DIM_PRODUCTS(PRODUCT_ID),
    TRANSACTIONS_CUSTOMERS as FACT_TRANSACTIONS(CUSTOMER_ID) references DIM_CUSTOMERS(CUSTOMER_ID),
    TRANSACTIONS_DATES as FACT_TRANSACTIONS(DATE_KEY) references DIM_DATES(DATE_KEY),
    TRANSACTIONS_PLATFORMS as FACT_TRANSACTIONS(PLATFORM_ID) references DIM_PLATFORMS(PLATFORM_ID),
    TRANSACTIONS_PRODUCTS as FACT_TRANSACTIONS(PRODUCT_ID) references DIM_PRODUCTS(PRODUCT_ID),
    CUSTOMER_REVIEWS_DATES as FACT_CUSTOMER_REVIEWS(DATE_KEY) references DIM_DATES(DATE_KEY),
    CUSTOMER_REVIEWS_PRODUCT_HIERARCHY as FACT_CUSTOMER_REVIEWS(CATEGORY_HIER_1_ID, CATEGORY_HIER_2_ID) references DIM_PRODUCT_HIERARCHY(CATEGORY_HIER_1_ID, CATEGORY_HIER_2_ID)
) facts (
    PUBLIC DIM_PRODUCTS.PRODUCT_UNIT_COGS as PRODUCT_UNIT_COGS with synonyms =(
        'unit_cost',
        'cost_per_unit',
        'product_cost',
        'unit_expense',
        'cost_of_goods_sold',
        'product_expense'
    ) comment = 'The cost of goods sold for a single unit of each product.',
    PUBLIC DIM_PRODUCTS.PRODUCT_UNIT_PRICE as PRODUCT_UNIT_PRICE with synonyms =(
        'unit_price',
        'price_per_unit',
        'item_price',
        'price_per_item'
    ) comment = 'The price of a single unit of a product.',
    PUBLIC FACT_DAILY_STOCK_LEVELS.AVAILABLE_STOCK as AVAILABLE_STOCK with synonyms =(
        'current_stock',
        'stock_balance',
        'stock_level',
        'inventory_level',
        'stock_on_hand',
        'total_stock',
        'stock_quantity'
    ) comment = 'The quantity of a particular stock item that is currently available for sale or use, reflecting the current level of inventory on hand.',
    PUBLIC FACT_DAILY_STOCK_LEVELS.STOCK_CHANGE as STOCK_CHANGE with synonyms =(
        'stock_difference',
        'stock_variation',
        'stock_fluctuation',
        'inventory_change',
        'stock_deviation',
        'stock_shift'
    ) comment = 'The daily change in the stock level, representing the difference between the current day''s stock level and the previous day''s stock level, with a negative value indicating a decrease and a positive value indicating an increase.',
    PUBLIC FACT_SUPPLIER_DELIVERIES.QUANTITY_DELIVERED as QUANTITY_DELIVERED with synonyms =(
        'supplier_amount',
        'quantity_supplied',
        'supplier_volume',
        'delivered_quantity',
        'supplied_units'
    ) comment = 'The quantity of products delivered by a supplier.',
    PUBLIC FACT_TRANSACTIONS.QUANTITY as QUANTITY with synonyms =(
        'amount',
        'count',
        'volume',
        'number',
        'total',
        'units',
        'items',
        'stock',
        'inventory',
        'bulk'
    ) comment = 'The quantity of items involved in a transaction.',
    PUBLIC FACT_TRANSACTIONS.SESSION_DURATION as SESSION_DURATION with synonyms =(
        'session_length',
        'interaction_time',
        'visit_duration',
        'time_spent',
        'engagement_time',
        'active_time'
    ) comment = 'The length of time, in seconds, that a user''s session lasted.',
    PUBLIC FACT_CUSTOMER_REVIEWS.REVIEW_LENGTH as REVIEW_LENGTH with synonyms =(
        'review_size',
        'review_word_count',
        'text_length',
        'character_count',
        'review_character_length',
        'content_length'
    ) comment = 'The length of the review text submitted by the customer.',
    PUBLIC FACT_CUSTOMER_REVIEWS.SENTIMENT_BRAND as SENTIMENT_BRAND with synonyms =(
        'brand_sentiment_score',
        'brand_opinion',
        'brand_feeling',
        'brand_attitude',
        'brand_appraisal',
        'brand_emotion',
        'brand_perception',
        'brand_view'
    ) comment = 'The sentiment of the customer''s review towards the brand, where 1 indicates a positive sentiment, -1 indicates a negative sentiment, and 0 indicates a neutral sentiment.',
    PUBLIC FACT_CUSTOMER_REVIEWS.SENTIMENT_OVERALL as SENTIMENT_OVERALL with synonyms =(
        'overall_sentiment',
        'overall_feeling',
        'general_sentiment',
        'sentiment_score',
        'overall_rating',
        'overall_opinion',
        'general_feeling',
        'overall_attitude'
    ) comment = 'The overall sentiment of the customer''s review, where 1 indicates a positive sentiment, -1 indicates a negative sentiment, and 0 indicates a neutral sentiment.',
    PUBLIC FACT_CUSTOMER_REVIEWS.SENTIMENT_PRICE as SENTIMENT_PRICE with synonyms =(
        'price_sentiment',
        'price_feeling',
        'price_opinion',
        'price_attitude',
        'price_appraisal',
        'price_assessment',
        'price_evaluation'
    ) comment = 'The sentiment of the customer''s review regarding the price of the product, where 1 indicates a positive sentiment, -1 indicates a negative sentiment, and 0 indicates a neutral sentiment.',
    PUBLIC FACT_CUSTOMER_REVIEWS.SENTIMENT_PRODUCT as SENTIMENT_PRODUCT with synonyms =(
        'product_opinion',
        'product_feeling',
        'product_attitude',
        'product_appraisal',
        'product_assessment',
        'product_rating',
        'product_evaluation'
    ) comment = 'The sentiment of the customer''s review regarding the product, where 1 indicates a positive sentiment, -1 indicates a negative sentiment, and 0 indicates a neutral sentiment.',
    PUBLIC FACT_CUSTOMER_REVIEWS.SENTIMENT_QUALITY as SENTIMENT_QUALITY with synonyms =(
        'product_rating',
        'quality_score',
        'item_quality',
        'product_evaluation',
        'quality_assessment',
        'product_satisfaction'
    ) comment = 'The sentiment of the customer''s review regarding the quality of the product or service, where 1 indicates a positive sentiment, -1 indicates a negative sentiment, and 0 indicates a neutral sentiment.'
) dimensions (
    PUBLIC DIM_CUSTOMERS.AGE_GROUP as CASE
        WHEN DATEDIFF('year', BIRTHDATE, CURRENT_DATE()) < 25 THEN 'Under 25'
        WHEN DATEDIFF('year', BIRTHDATE, CURRENT_DATE()) < 35 THEN '25-34'
        WHEN DATEDIFF('year', BIRTHDATE, CURRENT_DATE()) < 50 THEN '35-49'
        ELSE '50+'
    END with synonyms =(
        'age_range',
        'age_category',
        'age_bracket',
        'demographic_group',
        'age_segment'
    ) comment = 'Customer age group',
    PUBLIC DIM_CUSTOMERS.BIRTHDATE as BIRTHDATE with synonyms =(
        'date_of_birth',
        'birth_date',
        'dob',
        'birthday',
        'date_of_origin'
    ) comment = 'Date of birth of the customer.',
    PUBLIC DIM_CUSTOMERS.COUNTRY as COUNTRY with synonyms =(
        'nation',
        'land',
        'territory',
        'state',
        'region',
        'homeland',
        'nationality',
        'geographic_location',
        'place_of_origin'
    ) comment = 'The country where the customer is located.',
    PUBLIC DIM_CUSTOMERS.CUSTOMER_AGE as DATEDIFF('year', BIRTHDATE, CURRENT_DATE()) with synonyms =(
        'age',
        'customer_age_in_years',
        'years_old',
        'customer_birth_year',
        'customer_age_value'
    ) comment = 'Current age of customer',
    PUBLIC DIM_CUSTOMERS.CUSTOMER_ID as CUSTOMER_ID with synonyms =(
        'client_id',
        'customer_key',
        'account_number',
        'user_id',
        'client_number',
        'account_id',
        'customer_reference',
        'unique_customer_identifier'
    ) comment = 'Unique identifier for each customer in the database.',
    PUBLIC DIM_CUSTOMERS.GENDER as GENDER with synonyms =(
        'sex',
        'male_female',
        'demographic',
        'biological_sex',
        'person_type'
    ) comment = 'The gender of the customer, either Male or Female.',
    PUBLIC DIM_CUSTOMERS.NAME as NAME with synonyms =(
        'full_name',
        'customer_name',
        'person_name',
        'individual_name',
        'client_name'
    ) comment = 'The name of the customer.',
    PUBLIC DIM_DATES.DATE as DATE with synonyms =(
        'calendar_date',
        'date_value',
        'date_field',
        'calendar_day',
        'date_column',
        'date_key_date',
        'date_attribute'
    ) comment = 'Date dimension representing a specific point in time, used for tracking and analyzing data over a timeline.',
    PUBLIC DIM_DATES.DATE_KEY as DATE_KEY with synonyms =(
        'date_id',
        'date_code',
        'calendar_key',
        'date_identifier',
        'date_number',
        'calendar_id',
        'date_index'
    ) comment = 'Unique identifier for a specific date in the format YYYYMMDD.',
    PUBLIC DIM_DATES.DATE_TRUNCATED_TO_MONTH as DATE_TRUNCATED_TO_MONTH with synonyms =(
        'first_day_of_month',
        'month_start_date',
        'truncated_month',
        'month_trunc',
        'first_day_of_calendar_month',
        'month_beginning_date'
    ) comment = 'The date truncated to the first day of the month, effectively removing the day component and representing the month as a whole.',
    PUBLIC DIM_DATES.DATE_TRUNCATED_TO_WEEK as DATE_TRUNCATED_TO_WEEK with synonyms =(
        'week_start_date',
        'start_of_week',
        'week_beginning',
        'truncated_week',
        'weekly_start_date',
        'week_truncation',
        'beginning_of_week'
    ) comment = 'The date truncated to the first day of the week (Sunday) in which the original date falls.',
    PUBLIC DIM_DATES.DAY_NAME as DAY_NAME with synonyms =(
        'day_of_week',
        'weekday',
        'day_type',
        'day_description',
        'day_label'
    ) comment = 'The full name of the day of the week.',
    PUBLIC DIM_DATES.DAY_OF_YEAR as DAY_OF_YEAR with synonyms =(
        'day_of_calendar',
        'julian_day',
        'day_number',
        'annual_day',
        'yearly_day',
        'day_in_year'
    ) comment = 'The day of the year, ranging from 1 (January 1) to 365 (December 31), representing the sequential day of the year.',
    PUBLIC DIM_DATES.IS_CURRENT_MONTH as DATE_TRUNC('MONTH', DATE) = DATE_TRUNC('MONTH', CURRENT_DATE()) with synonyms =(
        'is_current',
        'current_month_flag',
        'is_this_month',
        'current_month_indicator',
        'is_month_current',
        'month_is_current'
    ) comment = 'Whether the date is in the current month',
    PUBLIC DIM_DATES.IS_WEEKEND as DAY_NAME IN ('Sat', 'Sun') with synonyms =('weekend_flag', 'weekend_indicator') comment = ' Whether the date falls on a weekend',
    PUBLIC DIM_DATES.MONTH as MONTH with synonyms =(
        'month_number',
        'month_value',
        'month_code',
        'calendar_month',
        'month_of_year'
    ) comment = 'The month of the year, represented as a numerical value (1-12), where 1 corresponds to January and 12 corresponds to December.',
    PUBLIC DIM_DATES.MONTH_NAME as MONTH_NAME with synonyms =(
        'month_full_name',
        'full_month_name',
        'month_long_name',
        'month_full_description',
        'month_text'
    ) comment = 'The full name of the month, such as January, February, or March, represented by a three-letter abbreviation.',
    PUBLIC DIM_DATES.QUARTER as QUARTER with synonyms =(
        'quarterly_period',
        'financial_quarter',
        'fiscal_quarter',
        'quarterly_term',
        'period_of_year'
    ) comment = 'The quarter of the year, with 1 representing January to March, 2 representing April to June, 3 representing July to September, and 4 (not listed) representing October to December.',
    PUBLIC DIM_DATES.SEASON as CASE
        when MONTH in (12, 1, 2) then 'Winter'
        when MONTH in (3, 4, 5) then 'Spring'
        when MONTH in (6, 7, 8) then 'Summer'
        else 'Fall'
    END with synonyms =('seasonal_period', 'time_of_year') comment = 'Season of the year',
    PUBLIC DIM_DATES.WEEK_OF_YEAR as WEEK_OF_YEAR with synonyms =(
        'week_number',
        'calendar_week',
        'week_of_calendar',
        'yearly_week_number',
        'week_in_year'
    ) comment = 'The week of the year, ranging from 1 to 52 or 53, depending on the year, indicating the week number in which a date falls.',
    PUBLIC DIM_DATES.YEAR as YEAR with synonyms =(
        'calendar_year',
        'fiscal_year',
        'annual_period',
        'yearly_period',
        'twelve_month_period'
    ) comment = 'The calendar year to which a date belongs.',
    PUBLIC DIM_PLATFORMS.PLATFORM_ID as PLATFORM_ID with synonyms =(
        'platform_key',
        'platform_code',
        'platform_identifier',
        'platform_reference',
        'platform_unique_id'
    ) comment = 'Unique identifier for a specific platform, used to distinguish between different platforms in the system.',
    PUBLIC DIM_PLATFORMS.PLATFORM_NAME as PLATFORM_NAME with synonyms =(
        'platform_title',
        'system_name',
        'software_name',
        'application_name',
        'service_name',
        'product_name'
    ) comment = 'The platform through which customers interact with the company, such as social media, video sharing, or the company''s own website.',
    PUBLIC DIM_PRODUCTS.CATEGORY_HIER_1_ID as CATEGORY_HIER_1_ID with synonyms =(
        'top_level_category_id',
        'primary_category_id',
        'category_level_1',
        'first_level_category',
        'high_level_category_id',
        'main_category_id'
    ) comment = 'The top-level category identifier for a product in the hierarchical product categorization structure.',
    PUBLIC DIM_PRODUCTS.CATEGORY_HIER_2_ID as CATEGORY_HIER_2_ID with synonyms =(
        'subcategory_id',
        'category_level_2',
        'secondary_category',
        'subcategory_code',
        'category_hierarchy_2',
        'second_level_category'
    ) comment = 'The second level of the product category hierarchy.',
    PUBLIC DIM_PRODUCTS.PRODUCT_ID as PRODUCT_ID with synonyms =(
        'product_key',
        'product_code',
        'item_id',
        'product_identifier',
        'sku',
        'product_number'
    ) comment = 'Unique identifier for a product in the catalog.',
    PUBLIC DIM_PRODUCTS.PRODUCT_NAME as PRODUCT_NAME with synonyms =(
        'product_title',
        'item_name',
        'product_description',
        'item_description',
        'product_label'
    ) comment = 'The name of the product being sold, which can be used to analyze sales trends and customer preferences by specific product.',
    PUBLIC DIM_PRODUCTS.PRODUCT_TIER as PRODUCT_TIER with synonyms =(
        'product_level',
        'product_category',
        'product_rank',
        'product_grade',
        'product_classification',
        'product_segment'
    ) comment = 'The PRODUCT_TIER dimension categorizes products into three distinct levels of quality and features, where Premium represents high-end products with advanced features, Standard represents mid-range products with standard features, and Budget represents entry-level products with basic features.',
    PUBLIC DIM_PRODUCT_HIERARCHY.CATEGORY_HIER_1_ID as CATEGORY_HIER_1_ID with synonyms =(
        'top_level_category_id',
        'primary_category_id',
        'category_group_id',
        'first_level_category_identifier',
        'high_level_category_code'
    ) comment = 'The top-level category identifier in the product hierarchy, representing the highest level of categorization for a product.',
    PUBLIC DIM_PRODUCT_HIERARCHY.CATEGORY_HIER_1_NAME as CATEGORY_HIER_1_NAME with synonyms =(
        'top_level_category',
        'primary_category',
        'main_category',
        'first_level_category',
        'high_level_category',
        'category_name_level_1',
        'parent_category',
        'category_1_name'
    ) comment = 'The top-level category name for a product in the hierarchy, representing the broadest classification of the product (e.g. Confectionery, Beverages, Food Products).',
    PUBLIC DIM_PRODUCT_HIERARCHY.CATEGORY_HIER_2_ID as CATEGORY_HIER_2_ID with synonyms =(
        'subcategory_id',
        'category_level_2_id',
        'subcategory_key',
        'secondary_category_id',
        'category_level_two_id',
        'subcat_id'
    ) comment = 'The second level of the product category hierarchy, which groups products into sub-categories within the overall product category.',
    PUBLIC DIM_PRODUCT_HIERARCHY.CATEGORY_HIER_2_NAME as CATEGORY_HIER_2_NAME with synonyms =(
        'subcategory_name',
        'subcategory_label',
        'category_level_2_name',
        'secondary_category_name',
        'subcategory_title',
        'second_level_category_name'
    ) comment = 'The second level of categorization for a product in the hierarchy, providing a more specific grouping of products within a category.',
    PUBLIC DIM_SUPPLIERS.CITY as CITY with synonyms =(
        'town',
        'municipality',
        'metropolis',
        'urban_area',
        'location',
        'municipality_name',
        'urban_center',
        'geographical_location'
    ) comment = 'The city where the supplier is located.',
    PUBLIC DIM_SUPPLIERS.CONTACT_PERSON as CONTACT_PERSON with synonyms =(
        'account_manager',
        'sales_representative',
        'point_of_contact',
        'key_contact',
        'main_contact',
        'supplier_representative',
        'liaison_person'
    ) comment = 'The name of the person who serves as the primary point of contact for the supplier organization.',
    PUBLIC DIM_SUPPLIERS.COUNTRY as COUNTRY with synonyms =('nation', 'region', 'location', 'territory') comment = 'The country where the supplier is located.',
    PUBLIC DIM_SUPPLIERS.STREET as STREET with synonyms =(
        'address',
        'road',
        'avenue',
        'route',
        'way',
        'thoroughfare',
        'residence',
        'building_location'
    ) comment = 'Street address of the supplier''s location.',
    PUBLIC DIM_SUPPLIERS.SUPPLIER_ID as SUPPLIER_ID with synonyms =(
        'supplier_key',
        'vendor_id',
        'provider_id',
        'supplier_code',
        'supplier_number',
        'vendor_key'
    ) comment = 'Unique identifier for a supplier in the system.',
    PUBLIC DIM_SUPPLIERS.SUPPLIER_NAME as SUPPLIER_NAME with synonyms =(
        'supplier_title',
        'vendor_name',
        'provider_name',
        'company_name',
        'organization_name',
        'business_name'
    ) comment = 'The name of the supplier company.',
    PUBLIC FACT_DAILY_STOCK_LEVELS.DATE_KEY as DATE_KEY with synonyms =(
        'date_id',
        'date_code',
        'calendar_date',
        'date_dimension',
        'time_key',
        'calendar_key',
        'date_reference'
    ) comment = 'Date key representing the date for which the stock level is recorded, in the format YYYYMMDD.',
    PUBLIC FACT_DAILY_STOCK_LEVELS.PRODUCT_ID as PRODUCT_ID with synonyms =(
        'product_code',
        'item_id',
        'product_number',
        'item_code',
        'article_id'
    ) comment = 'Unique identifier for a product in the inventory.',
    PUBLIC FACT_SUPPLIER_DELIVERIES.DATE_KEY as DATE_KEY with synonyms =(
        'date_id',
        'date_code',
        'calendar_key',
        'date_reference',
        'calendar_date_id',
        'date_identifier'
    ) comment = 'Date when the delivery was made, represented as an integer in the format YYYYMMDD.',
    PUBLIC FACT_SUPPLIER_DELIVERIES.PRODUCT_ID as PRODUCT_ID with synonyms =(
        'product_code',
        'item_id',
        'product_number',
        'item_code',
        'article_number',
        'product_key'
    ) comment = 'Unique identifier for the product being delivered.',
    PUBLIC FACT_SUPPLIER_DELIVERIES.SUPPLIER_ID as SUPPLIER_ID with synonyms =(
        'provider_id',
        'vendor_id',
        'supplier_code',
        'provider_code',
        'vendor_code',
        'source_id'
    ) comment = 'Unique identifier for the supplier who made the delivery.',
    PUBLIC FACT_TRANSACTIONS.CUSTOMER_ID as CUSTOMER_ID with synonyms =(
        'client_id',
        'customer_key',
        'account_id',
        'user_id',
        'patron_id',
        'buyer_id',
        'client_key',
        'account_number'
    ) comment = 'Unique identifier for the customer who made the transaction.',
    PUBLIC FACT_TRANSACTIONS.DATE_KEY as DATE_KEY with synonyms =(
        'date_id',
        'date_code',
        'calendar_key',
        'date_reference',
        'time_key',
        'calendar_id'
    ) comment = 'Date on which the transaction occurred, represented as an integer in the format YYYYMMDD.',
    PUBLIC FACT_TRANSACTIONS.PLATFORM_ID as PLATFORM_ID with synonyms =(
        'channel_id',
        'site_id',
        'medium_id',
        'source_id',
        'distribution_channel'
    ) comment = 'Unique identifier for the platform where the transaction took place.',
    PUBLIC FACT_TRANSACTIONS.PRODUCT_ID as PRODUCT_ID with synonyms =(
        'item_id',
        'product_code',
        'item_code',
        'product_number',
        'sku',
        'product_key'
    ) comment = 'Unique identifier for the product being transacted.',
    PUBLIC FACT_CUSTOMER_REVIEWS.CATEGORY_HIER_1_ID as CATEGORY_HIER_1_ID with synonyms =(
        'top_level_category_id',
        'primary_category_id',
        'category_id_level_1',
        'first_level_category',
        'high_level_category_id',
        'main_category_id'
    ) comment = 'The top-level category of the product being reviewed, as part of a hierarchical categorization system.',
    PUBLIC FACT_CUSTOMER_REVIEWS.CATEGORY_HIER_2_ID as CATEGORY_HIER_2_ID with synonyms =(
        'subcategory_id',
        'subcategory_code',
        'category_level_2',
        'secondary_category_id',
        'nested_category_id',
        'subcat_id',
        'category_subgroup_id'
    ) comment = 'Category Hierarchy Level 2 Identifier',
    PUBLIC FACT_CUSTOMER_REVIEWS.DATE_KEY as DATE_KEY with synonyms =(
        'date_id',
        'date_code',
        'calendar_key',
        'date_identifier',
        'timestamp_key',
        'calendar_date_id'
    ) comment = 'Date when the customer review was submitted, in the format YYYYMMDD.'
) metrics (
    PUBLIC FACT_DAILY_STOCK_LEVELS.TOTAL_STOCK as SUM(AVAILABLE_STOCK) with synonyms =(
        'total_inventory',
        'overall_stock',
        'stock_level',
        'total_quantity',
        'inventory_level',
        'stock_total',
        'total_amount_in_stock'
    ) comment = 'The total quantity of stock on hand for a particular item or product at the end of each day.',
    PUBLIC FACT_SUPPLIER_DELIVERIES.TOTAL_QUANTITY_DELIVERED as SUM(QUANTITY_DELIVERED) with synonyms =('delivery_quantity') comment = 'Total quantity of all delivered products',
    PUBLIC FACT_TRANSACTIONS.PROFIT as SUM(
        FACT_TRANSACTIONS.QUANTITY * (
            DIM_PRODUCTS.PRODUCT_UNIT_PRICE - DIM_PRODUCTS.PRODUCT_UNIT_COGS
        )
    ) with synonyms =('margin', 'gross_profit', 'profit_margin') comment = ' Total profit calculated as revenue minus cost of goods sold',
    PUBLIC FACT_TRANSACTIONS.REVENUE as SUM(
        FACT_TRANSACTIONS.QUANTITY * DIM_PRODUCTS.PRODUCT_UNIT_PRICE
    ) with synonyms =(
        'sales',
        'turnover',
        'income',
        'earnings',
        'profit',
        'proceeds',
        'receipts',
        'gain',
        'revenue'
    ) comment = 'The total amount of money earned from sales or services provided, representing the income generated by the business.',
    PUBLIC FACT_TRANSACTIONS.TOTAL_QUANTITY_SOLD as SUM(QUANTITY) with synonyms =(
        'total_items_sold',
        'total_units_sold',
        'quantity_sold',
        'total_volume_sold',
        'total_amount_sold',
        'items_sold',
        'units_sold',
        'sales_quantity',
        'sold_quantity'
    ) comment = 'Total quantity of products sold'
) with extension (
    CA = '{"tables":[{"name":"DIM_CUSTOMERS","dimensions":[{"name":"COUNTRY","cortex_search_service":{"database":"AI_DEVELOPMENT","schema":"SI_THE_FOOD_AND_BEVERAGE_COMPANY","service":"_LITERAL_SEARCH_CUSTOMER_COUNTRY"}},{"name":"CUSTOMER_ID","sample_values":["CID_00091","CID_00243","CID_00019"]},{"name":"GENDER","sample_values":["Male","Female"],"is_enum":true},{"name":"NAME","cortex_search_service":{"database":"AI_DEVELOPMENT","schema":"SI_THE_FOOD_AND_BEVERAGE_COMPANY","service":"_LITERAL_SEARCH_CUSTOMER_NAME"}},{"name":"AGE_GROUP","is_enum":true},{"name":"CUSTOMER_AGE"}],"time_dimensions":[{"name":"BIRTHDATE","sample_values":["1979-03-29","1978-05-09","1981-10-03"]}]},{"name":"DIM_DATES","dimensions":[{"name":"DATE_KEY","sample_values":["20240227","20240606","20240730"]},{"name":"DATE_TRUNCATED_TO_MONTH"},{"name":"DATE_TRUNCATED_TO_WEEK"},{"name":"DAY_NAME","sample_values":["Tue","Mon","Wed"]},{"name":"DAY_OF_YEAR","sample_values":["98","29","1"]},{"name":"MONTH","sample_values":["1","2","3"]},{"name":"MONTH_NAME","sample_values":["Jan","Mar","Feb"]},{"name":"QUARTER","sample_values":["1","2","3","4"],"is_enum":true},{"name":"WEEK_OF_YEAR","sample_values":["29","1","2"]},{"name":"YEAR","sample_values":["2024","2025"]},{"name":"IS_WEEKEND"},{"name":"SEASON","is_enum":true},{"name":"IS_CURRENT_MONTH"}],"time_dimensions":[{"name":"DATE","sample_values":["2024-01-12","2024-01-01","2024-04-18"]}]},{"name":"DIM_PLATFORMS","dimensions":[{"name":"PLATFORM_ID","sample_values":["PLATFORM_003","PLATFORM_002","PLATFORM_001"]},{"name":"PLATFORM_NAME","sample_values":["Instagram","YouTube","Website","Facebook","TikTok"],"is_enum":true}]},{"name":"DIM_PRODUCTS","dimensions":[{"name":"CATEGORY_HIER_1_ID","sample_values":["HIER_1","HIER_2","HIER_3"]},{"name":"CATEGORY_HIER_2_ID","sample_values":["HIER_1_1","HIER_5_1","HIER_2_1"]},{"name":"PRODUCT_ID","sample_values":["PID_00000033","PID_00000000","PID_00000001"]},{"name":"PRODUCT_NAME","sample_values":["Senior Dog Wellness Blend","Reserve French Roast Whole Bean","Gourmet Espresso Capsules Gold"],"cortex_search_service":{"database":"AI_DEVELOPMENT","schema":"SI_THE_FOOD_AND_BEVERAGE_COMPANY","service":"_LITERAL_SEARCH_PRODUCT_NAME"}},{"name":"PRODUCT_TIER","sample_values":["Premium","Standard","Budget"],"is_enum":true}],"facts":[{"name":"PRODUCT_UNIT_COGS","sample_values":["8.5","3.1","5.2"]},{"name":"PRODUCT_UNIT_PRICE","sample_values":["8.99","15.99","5.99"]}]},{"name":"DIM_PRODUCT_HIERARCHY","dimensions":[{"name":"CATEGORY_HIER_1_ID","sample_values":["HIER_1","HIER_2","HIER_3"]},{"name":"CATEGORY_HIER_1_NAME","sample_values":["Beverages","Confectionery","Food Products","Health & Nutrition","Pet Care"],"cortex_search_service":{"database":"AI_DEVELOPMENT","schema":"SI_THE_FOOD_AND_BEVERAGE_COMPANY","service":"_LITERAL_SEARCH_PRODUCT_HIERARCHY_1"}},{"name":"CATEGORY_HIER_2_ID","sample_values":["HIER_1_1","HIER_5_1","HIER_2_1"]},{"name":"CATEGORY_HIER_2_NAME","sample_values":["Nutritional supplements","Baby Food","Water","Chocolate bars","Cooking aids","Dairy","Medical nutrition products","Frozen and chilled foods","Pet food and treats","Premium Chocolate","Coffee","Breakfast cereals and bars","Performance nutrition"],"cortex_search_service":{"database":"AI_DEVELOPMENT","schema":"SI_THE_FOOD_AND_BEVERAGE_COMPANY","service":"_LITERAL_SEARCH_PRODUCT_HIERARCHY_2"}}]},{"name":"DIM_SUPPLIERS","dimensions":[{"name":"CITY","sample_values":["Munich","Lyon","London"]},{"name":"CONTACT_PERSON","sample_values":["Margaret Thompson","Klaus Mueller","Giuseppe Rossi"]},{"name":"COUNTRY","sample_values":["Germany","France","United Kingdom"]},{"name":"STREET","sample_values":["Maximilianstrasse 12","Baker Street 221B","Via Torino 45"]},{"name":"SUPPLIER_ID","sample_values":["SUP_ID_00001","SUP_ID_00002","SUP_ID_00000"]},{"name":"SUPPLIER_NAME","sample_values":["Premium Confectionery Ltd","Premium Coffee Roasters Co","Natural Springs Co"]}]},{"name":"FACT_DAILY_STOCK_LEVELS","dimensions":[{"name":"DATE_KEY","sample_values":["20240509","20240905","20250126"]},{"name":"PRODUCT_ID","sample_values":["PID_00000008","PID_00000107","PID_00000071"]}],"facts":[{"name":"AVAILABLE_STOCK","sample_values":["75","269","55"]},{"name":"STOCK_CHANGE","sample_values":["-33","128","-34"]}],"metrics":[{"name":"TOTAL_STOCK"}]},{"name":"FACT_SUPPLIER_DELIVERIES","dimensions":[{"name":"DATE_KEY","sample_values":["20250224","20250623","20250728"]},{"name":"PRODUCT_ID","sample_values":["PID_00000113","PID_00000057","PID_00000118"]},{"name":"SUPPLIER_ID","sample_values":["SUP_ID_00011","SUP_ID_00000","SUP_ID_00004"]}],"facts":[{"name":"QUANTITY_DELIVERED","sample_values":["20","30","10"]}],"metrics":[{"name":"TOTAL_QUANTITY_DELIVERED"}]},{"name":"FACT_TRANSACTIONS","dimensions":[{"name":"CUSTOMER_ID","sample_values":["CID_00058","CID_04188","CID_03715"]},{"name":"DATE_KEY","sample_values":["20240101","20240103","20240102"]},{"name":"PLATFORM_ID","sample_values":["PLATFORM_004","PLATFORM_002","PLATFORM_001"]},{"name":"PRODUCT_ID","sample_values":["PID_00000000","PID_00000001","PID_00000017"]}],"facts":[{"name":"QUANTITY","sample_values":["4","2","1"]},{"name":"SESSION_DURATION","sample_values":["250","247","258"]}],"metrics":[{"name":"REVENUE"},{"name":"PROFIT"},{"name":"TOTAL_QUANTITY_SOLD"}]},{"name":"FACT_CUSTOMER_REVIEWS","dimensions":[{"name":"CATEGORY_HIER_1_ID","sample_values":["HIER_4","HIER_5","HIER_3"]},{"name":"CATEGORY_HIER_2_ID","sample_values":["HIER_4_2","HIER_4_3","HIER_4_1"]},{"name":"DATE_KEY","sample_values":["20241030","20241228","20241023"]}],"facts":[{"name":"REVIEW_LENGTH","sample_values":["256","1120","552"]},{"name":"SENTIMENT_OVERALL","sample_values":["1","-1","0"]},{"name":"SENTIMENT_BRAND","sample_values":["1","-1","0"]},{"name":"SENTIMENT_PRICE","sample_values":["1","-1","0"]},{"name":"SENTIMENT_PRODUCT","sample_values":["1","-1","0"]},{"name":"SENTIMENT_QUALITY","sample_values":["1","-1","0"]}]}],"relationships":[{"name":"PRODUCTS_HIERARCHY"},{"name":"STOCK_DATES"},{"name":"STOCK_PRODUCTS"},{"name":"SUPPLIER_DATES"},{"name":"SUPPLIER_DIMENSIONS"},{"name":"SUPPLIER_PRODUCTS"},{"name":"TRANSACTIONS_CUSTOMERS"},{"name":"TRANSACTIONS_DATES"},{"name":"TRANSACTIONS_PLATFORMS"},{"name":"TRANSACTIONS_PRODUCTS"},{"name":"CUSTOMER_REVIEWS_DATES"},{"name":"CUSTOMER_REVIEWS_PRODUCT_HIERARCHY"}],"verified_queries":[{"name":"What was the weekly sales revenue for breakfast products?","question":"What was the weekly sales revenue for breakfast products?","sql":"WITH weekly_revenue AS (\\n  SELECT\\n    dd.date_truncated_to_week AS week_start,\\n    MIN(dd.date) AS start_date,\\n    MAX(dd.date) AS end_date,\\n    SUM(ft.quantity * dp.product_unit_price) AS weekly_revenue\\n  FROM\\n    fact_transactions AS ft\\n    LEFT OUTER JOIN dim_dates AS dd ON ft.date_key = dd.date_key\\n    LEFT OUTER JOIN dim_products AS dp ON ft.product_id = dp.product_id\\n    LEFT OUTER JOIN dim_product_hierarchy AS dph ON dp.category_hier_1_id = dph.category_hier_1_id\\n    AND dp.category_hier_2_id = dph.category_hier_2_id\\n  WHERE\\n    dph.category_hier_2_name = ''Breakfast cereals and bars''\\n  GROUP BY\\n    dd.date_truncated_to_week\\n)\\nSELECT\\n  week_start,\\n  MIN(start_date) AS period_start_date,\\n  MAX(end_date) AS period_end_date,\\n  weekly_revenue\\nFROM\\n  weekly_revenue\\nGROUP BY\\n  week_start,\\n  weekly_revenue\\nORDER BY\\n  week_start DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755070099},{"name":"What was the total quantity sold of beverages, their stock levels and quantity of supplier deliveries in Q1 2025 per calendar week?","question":"What was the total quantity sold of beverages, their stock levels and quantity of supplier deliveries in Q1 2025 per calendar week?","sql":"WITH weekly_sales AS (\\n  SELECT\\n    d.date_truncated_to_week AS week,\\n    SUM(ft.quantity) AS total_quantity_sold\\n  FROM\\n    fact_transactions AS ft\\n    LEFT OUTER JOIN dim_dates AS d ON ft.date_key = d.date_key\\n    LEFT OUTER JOIN dim_products AS dp ON ft.product_id = dp.product_id\\n    LEFT OUTER JOIN dim_product_hierarchy AS dph ON dp.category_hier_1_id = dph.category_hier_1_id\\n    AND dp.category_hier_2_id = dph.category_hier_2_id\\n  WHERE\\n    d.year = 2025\\n    AND d.quarter = 1\\n    AND dph.category_hier_1_name = ''Beverages''\\n  GROUP BY\\n    d.date_truncated_to_week\\n),\\nweekly_stock AS (\\n  SELECT\\n    d.date_truncated_to_week AS week,\\n    SUM(fdsl.available_stock) AS total_stock_level\\n  FROM\\n    fact_daily_stock_levels AS fdsl\\n    LEFT OUTER JOIN dim_dates AS d ON fdsl.date_key = d.date_key\\n    LEFT OUTER JOIN dim_products AS dp ON fdsl.product_id = dp.product_id\\n    LEFT OUTER JOIN dim_product_hierarchy AS dph ON dp.category_hier_1_id = dph.category_hier_1_id\\n    AND dp.category_hier_2_id = dph.category_hier_2_id\\n  WHERE\\n    d.year = 2025\\n    AND d.quarter = 1\\n    AND dph.category_hier_1_name = ''Beverages''\\n  GROUP BY\\n    d.date_truncated_to_week\\n),\\nweekly_deliveries AS (\\n  SELECT\\n    d.date_truncated_to_week AS week,\\n    SUM(fsd.quantity_delivered) AS total_quantity_delivered\\n  FROM\\n    fact_supplier_deliveries AS fsd\\n    LEFT OUTER JOIN dim_dates AS d ON fsd.date_key = d.date_key\\n    LEFT OUTER JOIN dim_products AS dp ON fsd.product_id = dp.product_id\\n    LEFT OUTER JOIN dim_product_hierarchy AS dph ON dp.category_hier_1_id = dph.category_hier_1_id\\n    AND dp.category_hier_2_id = dph.category_hier_2_id\\n  WHERE\\n    d.year = 2025\\n    AND d.quarter = 1\\n    AND dph.category_hier_1_name = ''Beverages''\\n  GROUP BY\\n    d.date_truncated_to_week\\n)\\nSELECT\\n  COALESCE(ws.week, wst.week, wd.week) AS week,\\n  COALESCE(ws.total_quantity_sold, 0) AS total_quantity_sold,\\n  COALESCE(wst.total_stock_level, 0) AS total_stock_level,\\n  COALESCE(wd.total_quantity_delivered, 0) AS total_quantity_delivered\\nFROM\\n  weekly_sales AS ws FULL\\n  OUTER JOIN weekly_stock AS wst ON ws.week = wst.week FULL\\n  OUTER JOIN weekly_deliveries AS wd ON COALESCE(ws.week, wst.week) = wd.week\\nORDER BY\\n  ws.week DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755070951},{"name":"What was the average sentiment regarding product quality peer week between january and april 2024 for dairy products?","question":"What was the average sentiment regarding product quality peer week between january and april 2024 for dairy products?","sql":"SELECT\\n  d.date_truncated_to_week AS week,\\n  AVG(fcr.sentiment_quality) AS avg_quality_sentiment\\nFROM\\n  fact_customer_reviews AS fcr\\n  LEFT OUTER JOIN dim_dates AS d ON fcr.date_key = d.date_key\\n  LEFT OUTER JOIN dim_product_hierarchy AS dph ON fcr.category_hier_1_id = dph.category_hier_1_id\\n  AND fcr.category_hier_2_id = dph.category_hier_2_id\\nWHERE\\n  d.year = 2024\\n  AND d.month BETWEEN 1\\n  AND 4\\n  AND dph.category_hier_2_name = ''Dairy''\\nGROUP BY\\n  d.date_truncated_to_week\\nORDER BY\\n  d.date_truncated_to_week DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755604197},{"name":"Which customer age groups generate the highest profit margins across different platforms?","question":"Which customer age groups generate the highest profit margins across different platforms?","sql":"WITH transaction_metrics AS (\\n  SELECT\\n    c.age_group,\\n    p.platform_name,\\n    MIN(d.date) AS start_date,\\n    MAX(d.date) AS end_date,\\n    SUM(\\n      t.quantity * (prod.product_unit_price - prod.product_unit_cogs)\\n    ) AS total_profit,\\n    SUM(t.quantity * prod.product_unit_price) AS total_revenue\\n  FROM\\n    fact_transactions AS t\\n    LEFT OUTER JOIN dim_customers AS c ON t.customer_id = c.customer_id\\n    LEFT OUTER JOIN dim_platforms AS p ON t.platform_id = p.platform_id\\n    LEFT OUTER JOIN dim_products AS prod ON t.product_id = prod.product_id\\n    LEFT OUTER JOIN dim_dates AS d ON t.date_key = d.date_key\\n  GROUP BY\\n    c.age_group,\\n    p.platform_name\\n),\\nprofit_margins AS (\\n  SELECT\\n    age_group,\\n    platform_name,\\n    start_date,\\n    end_date,\\n    total_profit,\\n    total_revenue,\\n    CASE\\n      WHEN total_revenue > 0 THEN total_profit / NULLIF(total_revenue, 0)\\n      ELSE NULL\\n    END AS profit_margin\\n  FROM\\n    transaction_metrics\\n)\\nSELECT\\n  age_group,\\n  platform_name,\\n  start_date,\\n  end_date,\\n  total_profit,\\n  total_revenue,\\n  profit_margin\\nFROM\\n  profit_margins\\nWHERE\\n  NOT age_group IS NULL\\n  AND NOT platform_name IS NULL\\nORDER BY\\n  profit_margin DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755606321},{"name":"How do beverage sales patterns differ by season, and which age groups drive summer vs winter consumption?","question":"How do beverage sales patterns differ by season, and which age groups drive summer vs winter consumption?","sql":"WITH seasonal_beverage_sales AS (\\n  SELECT\\n    d.season,\\n    c.age_group,\\n    MIN(d.date) AS start_date,\\n    MAX(d.date) AS end_date,\\n    SUM(ft.quantity) AS total_quantity_sold,\\n    SUM(ft.quantity * dp.product_unit_price) AS total_revenue\\n  FROM\\n    fact_transactions AS ft\\n    LEFT OUTER JOIN dim_dates AS d ON ft.date_key = d.date_key\\n    LEFT OUTER JOIN dim_customers AS c ON ft.customer_id = c.customer_id\\n    LEFT OUTER JOIN dim_products AS dp ON ft.product_id = dp.product_id\\n    LEFT OUTER JOIN dim_product_hierarchy AS dph ON dp.category_hier_1_id = dph.category_hier_1_id\\n    AND dp.category_hier_2_id = dph.category_hier_2_id\\n  WHERE\\n    dph.category_hier_1_name = ''Beverages''\\n    AND NOT d.season IS NULL\\n    AND NOT c.age_group IS NULL\\n  GROUP BY\\n    d.season,\\n    c.age_group\\n),\\nseason_totals AS (\\n  SELECT\\n    season,\\n    SUM(total_quantity_sold) AS season_total_quantity,\\n    SUM(total_revenue) AS season_total_revenue\\n  FROM\\n    seasonal_beverage_sales\\n  GROUP BY\\n    season\\n)\\nSELECT\\n  sbs.season,\\n  sbs.age_group,\\n  sbs.start_date,\\n  sbs.end_date,\\n  sbs.total_quantity_sold,\\n  sbs.total_revenue,\\n  st.season_total_quantity,\\n  st.season_total_revenue,\\n  CASE\\n    WHEN st.season_total_quantity > 0 THEN sbs.total_quantity_sold / NULLIF(NULLIF(st.season_total_quantity, 0), 0)\\n    ELSE NULL\\n  END AS age_group_quantity_share,\\n  CASE\\n    WHEN st.season_total_revenue > 0 THEN sbs.total_revenue / NULLIF(NULLIF(st.season_total_revenue, 0), 0)\\n    ELSE NULL\\n  END AS age_group_revenue_share\\nFROM\\n  seasonal_beverage_sales AS sbs\\n  LEFT OUTER JOIN season_totals AS st ON sbs.season = st.season\\nORDER BY\\n  sbs.season,\\n  sbs.total_quantity_sold DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755606622},{"name":"Which budget vs premium vs standard tier products have the best profit margins, and how does customer sentiment vary by tier?","question":"Which budget vs premium vs standard tier products have the best profit margins, and how does customer sentiment vary by tier?","sql":"WITH profit_metrics AS (\\n  SELECT\\n    prod.product_tier,\\n    MIN(d.date) AS start_date,\\n    MAX(d.date) AS end_date,\\n    SUM(\\n      t.quantity * (prod.product_unit_price - prod.product_unit_cogs)\\n    ) AS total_profit,\\n    SUM(t.quantity * prod.product_unit_price) AS total_revenue\\n  FROM\\n    fact_transactions AS t\\n    LEFT OUTER JOIN dim_products AS prod ON t.product_id = prod.product_id\\n    LEFT OUTER JOIN dim_dates AS d ON t.date_key = d.date_key\\n  GROUP BY\\n    prod.product_tier\\n),\\nprofit_margins AS (\\n  SELECT\\n    product_tier,\\n    start_date,\\n    end_date,\\n    total_profit,\\n    total_revenue,\\n    CASE\\n      WHEN total_revenue > 0 THEN total_profit / NULLIF(NULLIF(total_revenue, 0), 0)\\n      ELSE NULL\\n    END AS profit_margin\\n  FROM\\n    profit_metrics\\n),\\nsentiment_metrics AS (\\n  SELECT\\n    ph.category_hier_1_name,\\n    ph.category_hier_2_name,\\n    AVG(cr.sentiment_overall) AS avg_sentiment_overall,\\n    AVG(cr.sentiment_brand) AS avg_sentiment_brand,\\n    AVG(cr.sentiment_price) AS avg_sentiment_price,\\n    AVG(cr.sentiment_product) AS avg_sentiment_product,\\n    AVG(cr.sentiment_quality) AS avg_sentiment_quality\\n  FROM\\n    fact_customer_reviews AS cr\\n    LEFT OUTER JOIN dim_product_hierarchy AS ph ON cr.category_hier_1_id = ph.category_hier_1_id\\n    AND cr.category_hier_2_id = ph.category_hier_2_id\\n  GROUP BY\\n    ph.category_hier_1_name,\\n    ph.category_hier_2_name\\n),\\nproduct_sentiment AS (\\n  SELECT\\n    prod.product_tier,\\n    AVG(sm.avg_sentiment_overall) AS avg_sentiment_overall,\\n    AVG(sm.avg_sentiment_brand) AS avg_sentiment_brand,\\n    AVG(sm.avg_sentiment_price) AS avg_sentiment_price,\\n    AVG(sm.avg_sentiment_product) AS avg_sentiment_product,\\n    AVG(sm.avg_sentiment_quality) AS avg_sentiment_quality\\n  FROM\\n    dim_products AS prod\\n    LEFT OUTER JOIN dim_product_hierarchy AS ph ON prod.category_hier_1_id = ph.category_hier_1_id\\n    AND prod.category_hier_2_id = ph.category_hier_2_id\\n    LEFT OUTER JOIN sentiment_metrics AS sm ON ph.category_hier_1_name = sm.category_hier_1_name\\n    AND ph.category_hier_2_name = sm.category_hier_2_name\\n  GROUP BY\\n    prod.product_tier\\n)\\nSELECT\\n  pm.product_tier,\\n  pm.start_date,\\n  pm.end_date,\\n  pm.total_profit,\\n  pm.total_revenue,\\n  pm.profit_margin,\\n  ps.avg_sentiment_overall,\\n  ps.avg_sentiment_brand,\\n  ps.avg_sentiment_price,\\n  ps.avg_sentiment_product,\\n  ps.avg_sentiment_quality\\nFROM\\n  profit_margins AS pm\\n  LEFT OUTER JOIN product_sentiment AS ps ON pm.product_tier = ps.product_tier\\nWHERE\\n  NOT pm.product_tier IS NULL\\nORDER BY\\n  pm.profit_margin DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Michael Gorkow","verified_at":1755606686}]}'
);

## 1.5 Querying Semantic Views
Semantic Views can not only be used with Cortex Analyst but you can also query them with SQL.

This ensures a single authoritative definition with proper aggregation behavior. For example, when a user asks for “Revenue by Product Category,” the semantic view knows to aggregate at the appropriate level, whereas, without semantic views, dozens of inconsistent calculations might exist throughout different reports and applications, often with incorrect aggregation methods leading to erroneous results.

In [ ]:
SELECT * FROM SEMANTIC_VIEW(
    AI_DEVELOPMENT.SI_THE_FOOD_AND_BEVERAGE_COMPANY.SALES_MODEL
    DIMENSIONS 
      CATEGORY_HIER_1_NAME, 
      CATEGORY_HIER_2_NAME,
      YEAR,
      QUARTER
    METRICS 
      REVENUE,
      PROFIT
  )
  WHERE 
    CATEGORY_HIER_1_NAME = 'Beverages' AND
    YEAR = 2025
  ORDER BY 
    CATEGORY_HIER_2_NAME,
    YEAR,
    QUARTER;

## 1.5 Row Level Access Policies

In [ ]:
role_mappings = [
    ['Beverages', 'HIER_1', ['MANAGER_BEVERAGES']],
    ['Confectionery', 'HIER_2', ['MANAGER_CONFECTIONERY']],
    ['Food Products', 'HIER_3', ['MANAGER_FOOD']],
    ['Health & Nutrition', 'HIER_4', ['MANAGER_HEALTH_NUTRUTION']],
    ['Pet Care', 'HIER_5', ['MANAGER_PET_CARE']]
]
role_mappings = pd.DataFrame(role_mappings, columns=['ALLOWED_PRODUCT_CATEGORIES', 'ALLOWED_PRODUCT_CATEGORY_IDS', 'ALLOWED_ROLES'])
role_mappings = session.write_pandas(role_mappings, table_name='__RAP_PRODUCT_CATEGORY_ROLE_MAPPINGS', auto_create_table=True, overwrite=True, use_logical_type=True)
role_mappings.show()

In [ ]:
-- Drop any existing Row Level Access Policies
ALTER TABLE dim_products DROP ALL ROW ACCESS POLICIES;
ALTER TABLE dim_product_hierarchy DROP ALL ROW ACCESS POLICIES;

-- Create Row Level Access Policy
CREATE OR REPLACE ROW ACCESS POLICY RAP_PRODUCT_CATEGORY_ID
    AS (product_category_id varchar) RETURNS BOOLEAN ->
    'AI_ENGINEER' = current_role() -- allow access to all rows for AI_ENGINEER
    OR EXISTS (
        SELECT * FROM __RAP_PRODUCT_CATEGORY_ROLE_MAPPINGS
            WHERE
                ARRAY_CONTAINS(current_role()::VARIANT, as_array(ALLOWED_ROLES)) AND
                ALLOWED_PRODUCT_CATEGORY_IDS = product_category_id
        );


-- Add Row Level Access Policies to product tables
ALTER TABLE dim_products
  ADD ROW ACCESS POLICY RAP_PRODUCT_CATEGORY_ID
  ON (CATEGORY_HIER_1_ID);

ALTER TABLE dim_product_hierarchy
  ADD ROW ACCESS POLICY RAP_PRODUCT_CATEGORY_ID
  ON (CATEGORY_HIER_1_ID);

## 1.5.1 Testing Row Level Access Policies

In [ ]:
-- Query the semantic view as the manager for Confectionery
USE ROLE MANAGER_CONFECTIONERY;

SELECT * FROM SEMANTIC_VIEW(
    AI_DEVELOPMENT.SI_THE_FOOD_AND_BEVERAGE_COMPANY.SALES_MODEL
    DIMENSIONS 
      CATEGORY_HIER_1_NAME, 
      CATEGORY_HIER_2_NAME,
      YEAR,
      QUARTER
    METRICS 
      REVENUE
  )
  WHERE 
    YEAR = 2025
  ORDER BY 
    CATEGORY_HIER_2_NAME,
    YEAR,
    QUARTER;

In [ ]:
-- Query the semantic view as the AI_ENGINEER
USE ROLE AI_ENGINEER;

SELECT * FROM SEMANTIC_VIEW(
    AI_DEVELOPMENT.SI_THE_FOOD_AND_BEVERAGE_COMPANY.SALES_MODEL
    DIMENSIONS 
      CATEGORY_HIER_1_NAME, 
      CATEGORY_HIER_2_NAME,
      YEAR,
      QUARTER
    METRICS 
      REVENUE
  )
  WHERE 
    YEAR = 2025
  ORDER BY 
    CATEGORY_HIER_2_NAME,
    YEAR,
    QUARTER;